In [2]:
# Importeer benodigde bibliotheken
import pandas as pd
import openpyxl

# Inladen van de datasets
df = pd.read_excel('Consumptie_kwartierdata_2023.xlsx', skiprows=[0])
mp = pd.read_csv('marktprijzen_2023.csv')

# Dataset schoonmaken
nan_df = df.isna()
df_clean = df.fillna(0)

# Powerfactor toepassen
columns_to_multiply = ['Blindverbruik Consumptie (kVARh)', 'Blindverbruik Consumptie (kVARh).1', 'Blindverbruik Consumptie (kVARh).2']
factor = 0.8

kwh_df = df_clean

for column in columns_to_multiply:
    kwh_df[column] = kwh_df[column] * factor

# Optellen van verbruik
Total_consumptie1 = kwh_df['Elektriciteit Consumptie (kWh)'].sum()
Total_consumptie2 = kwh_df['Elektriciteit Consumptie (kWh).1'].sum()
Total_consumptie3 = kwh_df['Elektriciteit Consumptie (kWh).2'].sum()
Total_kVARH1 = kwh_df['Blindverbruik Consumptie (kVARh)'].sum()
Total_kVARH2 = kwh_df['Blindverbruik Consumptie (kVARh).1'].sum()
Total_kVARH3 = kwh_df['Blindverbruik Consumptie (kVARh).2'].sum()

total_consumptie = Total_consumptie1 + Total_consumptie2 + Total_consumptie3
total_kVARH = Total_kVARH1 + Total_kVARH2 + Total_kVARH3

print(f'Total consumptie: {total_consumptie}')
print(f'Total kVARH: {total_kVARH}')

# Berekenen van opslagkosten
opslag_kosten = 1496928.8992 * (0.00857 + 0.006)
print(f'Opslagkosten: {opslag_kosten}')

# Joinen van de datasets
# Zorg ervoor dat je de kolommen in beide datasets hebt die je nodig hebt voor de join
full_df = kwh_df.join(mp, how='inner', lsuffix='kwh_df', rsuffix='mp')

# Onnodige kolommen verwijderen
full_df = full_df.drop('StartDatemp', axis=1)

# Berekenen van de prijzen
price_df = full_df[['Elektriciteit Consumptie (kWh)', 'Blindverbruik Consumptie (kVARh)', 'Elektriciteit Consumptie (kWh).1', 'Blindverbruik Consumptie (kVARh).1', 'Elektriciteit Consumptie (kWh).2', 'Blindverbruik Consumptie (kVARh).2']].multiply(full_df["Price_kWh"], axis="index")
price_df['StartDatekwh_df'] = full_df['StartDatekwh_df']

first_column = price_df.pop('StartDatekwh_df')
price_df.insert(0, 'StartDatekwh_df', first_column)

# Sommeren van de columns om totalen te krijgen
Total_consumptie1 = price_df['Elektriciteit Consumptie (kWh)'].sum()
Total_consumptie2 = price_df['Elektriciteit Consumptie (kWh).1'].sum()
Total_consumptie3 = price_df['Elektriciteit Consumptie (kWh).2'].sum()

Total_kVARH1 = price_df['Blindverbruik Consumptie (kVARh)'].sum()
Total_kVARH2 = price_df['Blindverbruik Consumptie (kVARh).1'].sum()
Total_kVARH3 = price_df['Blindverbruik Consumptie (kVARh).2'].sum()

prijs_totaal = (Total_consumptie1 + Total_consumptie2 + Total_consumptie3 + Total_kVARH1 + Total_kVARH2 + Total_kVARH3) + opslag_kosten

prijs_round = prijs_totaal.round(2)
print(f'In het jaar 2023 met een marktprijscontract zouden de totale kosten voor Consumptie: € {prijs_round} zijn geweest')


Total consumptie: 939383.5
Total kVARH: 557545.3992
Opslagkosten: 21810.254061344


KeyError: "['StartDatemp'] not found in axis"